<a href="https://colab.research.google.com/github/tingshengxie/MATH-509/blob/main/Jay_Newby's_Lightweight_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving Particle_Tracking_Training_Data.py to Particle_Tracking_Training_Data.py


{'Particle_Tracking_Training_Data.py': b'import numpy as np\nfrom numpy import pi\nimport tensorflow as tf\n\nclass Particle_Tracking_Training_Data(tf.Module):\n    def __init__(self, Nt, rings=True):\n        self.Nt = int(Nt)\n        self.Ny = self.Nx = 256\n        self.d = 3\n        ximg = [[[i, j] for i in np.arange(self.Ny)]\n            for j in np.arange(self.Nx)]\n        self.ximg = np.float32(ximg)\n\n        x = np.arange(self.Nx) - self.Nx//2\n        y = np.arange(self.Ny) - self.Ny//2\n        X0, Y0 = np.meshgrid(x, y)\n        self.X = np.float32(X0)\n        self.Y = np.float32(Y0)\n\n        if rings:\n            self.ring_indicator = 1.\n        else:\n            self.ring_indicator = 0.\n\n        self._gen_video = tf.function(\n            input_signature=(\n                tf.TensorSpec(\n                    shape=[self.Ny, self.Nx, self.Nt, None], dtype=tf.float32),\n                tf.TensorSpec(shape=[self.Nt, None], dtype=tf.float32),\n                tf.

In [ ]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from ipywidgets import interact
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt
from tensorflow.keras.utils import register_keras_serializable

import Particle_Tracking_Training_Data

Populating the interactive namespace from numpy and matplotlib


In [ ]:
tf.random.set_seed(42)

In [ ]:
def build_multi_kernel_conv_model():

    # Two inputs images, one is the current frame image, the other is the previous frame image
    input_current = tf.keras.Input(shape=(256, 256, 1))

    input_previous = tf.keras.Input(shape=(256, 256, 1))

    # In the following two layers, we have several shared branches for both inputs.

    #######################################################################################################
    ###########################################   Layer 1 #################################################
    #######################################################################################################

    # 分支1：9×9 卷积核，输出通道设为 3（可根据需求调整）
    branch1_share = layers.Conv2D(filters=3,
                            kernel_size=(9, 9),
                            strides=2,
                            padding='same',
                            activation='elu')
    # 分支2：5×5 卷积核，输出通道设为 3
    branch2_share = layers.Conv2D(filters=3,
                            kernel_size=(5, 5),
                            strides=2,
                            padding='same',
                            activation='elu')

    # 分支3：3×3 卷积核，输出通道设为 3
    branch3_share = tf.keras.Sequential([
        layers.Conv2D(filters=3, kernel_size=(3, 3), strides=2, padding='same', activation='elu'),
        layers.MaxPooling2D(pool_size=3, strides=1, padding='same')
    ])

    # Applying sharing branches to both inputs

    # Current input image in layer 1
    conv1_current = branch1_share(input_current)
    conv2_current = branch2_share(input_current)
    conv3_current = branch3_share(input_current)

    # Previous input image in layer 1
    conv1_previous = branch1_share(input_previous)
    conv2_previous = branch2_share(input_previous)
    conv3_previous = branch3_share(input_previous)

    outputs1_current = layers.Concatenate(axis=-1)([conv1_current, conv2_current, conv3_current])
    outputs1_previous = layers.Concatenate(axis=-1)([conv1_previous, conv2_previous, conv3_previous])

    #######################################################################################################
    ###########################################   Layer 2 #################################################
    #######################################################################################################

    inputs2_current = outputs1_current
    inputs2_previous = outputs1_previous

    #branch 4: 7*7 convolution kernel, channels = 6
    branch4_share = layers.Conv2D(filters=6,
                            kernel_size=(7, 7),
                            strides=1,
                            padding='same',
                            activation='elu')

    #branch 5: 3*3 convolution kernel, channels = 6
    branch5 = layers.Conv2D(filters=6,
                            kernel_size=(3, 3),
                            strides=1,
                            padding='same',
                            activation='elu')

    # Applying layer and shared layer to both inputs
    conv4_current = branch4_share(inputs2_current)
    conv5_current = branch5(inputs2_current)

    conv4_previous = branch4_share(inputs2_previous)

    #######################################################################################################
    ########################################    Recurrent layer   #########################################
    #######################################################################################################

    # Recurrent layer
    branch_recurrent = tf.keras.Sequential([
        layers.Conv2D(filters=6, kernel_size=(7, 7), strides=1, dilation_rate=2, padding='same', activation='elu'),
        layers.Conv2D(filters=6, kernel_size=(7, 7), strides=1, dilation_rate=3, padding='same', activation='elu')
    ])

    # Applying the recurrent layer to previous input
    conv_recurrent = branch_recurrent(conv4_previous)


    # Output for layer 2, output channel = 18
    outputs2 = layers.Concatenate(axis=-1)([conv4_current, conv5_current, conv_recurrent])

    #######################################################################################################
    ###########################################   Layer 3 #################################################
    #######################################################################################################

    # Fully connected layer with 5×5 kernel, finalizing to 2 output channels (background vs particle)
    final_layer = layers.Conv2D(filters=2, kernel_size=(5, 5), padding='same', activation='elu')

    # Apply final layer
    conv6 = final_layer(outputs2)


    # Final output, applying softmax
    outputs = layers.Softmax(axis=1, name="prob_map")(conv6)


    # 构建模型
    model = Model(inputs=[input_current, input_previous],
                  outputs=outputs,
                  name="multi_kernel_conv")
    return model

In [ ]:
Nt = 100 ## number of frames for each video
kappa = 0.1 ## standard deviation of background noise added to image
a = 3. ## scale factor for the size of particle spots (not true size of particles)
IbackLevel = 0.15 ## relative intensity of randomly generated background pattern; in (0, 1)
Nparticles = 10 ## the number of particles (more => slower)
sigma_motion = 2.3 ## the standard deviation for particle brownian motion; should be in (0, 10)

## you might consider randomizing some of these parameters when training a neural net

pt = Particle_Tracking_Training_Data.Particle_Tracking_Training_Data(Nt) ## create object instance
## you can 'call' the object as many times as you want
## in this example, we only generate one training example
vid, labels, tracks = pt(kappa, a, IbackLevel, Nparticles, sigma_motion)

In [ ]:
# 创建并查看模型结构
model = build_multi_kernel_conv_model()
model.summary()

# 2. 生成一个随机输入 (batch_size=1, 256×256, 通道数=1)
frame_0 = tf.expand_dims(tf.expand_dims(vid[0], axis=0), axis=-1)
frame_1 = tf.expand_dims(tf.expand_dims(vid[1], axis=0), axis=-1)
y1 = model([frame_0, frame_1])
print("输入形状:", frame_0.shape)
print("输出形状:", y1.shape)

Model: "multi_kernel_conv"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 256, 256, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 3)    │            246 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 128, 3)    │             78 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 128, 128, 3)    │             30 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 128, 128, 9)    │              0 │ conv2d[0][0],          │
│                           │                        │                │ conv2d_1[0][0],        │
│                           │                        │                │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 128, 128, 9)    │              0 │ conv2d[1][0],          │
│ (Concatenate)             │                        │                │ conv2d_1[1][0],        │
│                           │                        │                │ sequential[1][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 6)    │          2,652 │ concatenate[0][0],     │
│                           │                        │                │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 128, 128, 6)    │            492 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_1 (Sequential) │ (None, 128, 128, 6)    │          3,540 │ conv2d_3[1][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 128, 128, 18)   │              0 │ conv2d_3[0][0],        │
│ (Concatenate)             │                        │                │ conv2d_4[0][0],        │
│                           │                        │                │ sequential_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 128, 128, 2)    │            902 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ prob_map (Softmax)        │ (None, 128, 128, 2)    │              0 │ conv2d_7[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,940 (31.02 KB)

 Trainable params: 7,940 (31.02 KB)

 Non-trainable params: 0 (0.00 B)

输入形状: (1, 256, 256, 1)
输出形状: (1, 128, 128, 2)


In [ ]:
X_curr = []
X_prev = []
Y_labels = []

for i in range(1, Nt):
    # 当前帧 shape (256,256,1)
    curr = tf.expand_dims(vid[i],   axis=-1)
    # 前一帧 shape (256,256,1)
    prev = tf.expand_dims(vid[i-1], axis=-1)

    X_curr.append(curr)
    X_prev.append(prev)

    Y_labels.append(labels[i])  # 对应当前帧的标签

X_curr = tf.stack(X_curr, axis=0)  # (Nt-1, 256,256,1)
X_prev = tf.stack(X_prev, axis=0)  # (Nt-1, 256,256,1)
Y_labels = tf.stack(Y_labels, axis=0)

In [ ]:
class OneHotMeanIoU(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        # total_confusion_matrix 的形状为 [num_classes, num_classes]
        self.total_cm = self.add_weight(
            name='total_confusion_matrix',
            shape=(num_classes, num_classes),
            initializer='zeros',  # 或 tf.zeros_initializer()
            dtype=tf.float32
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        """
        y_true, y_pred: (batch, height, width, num_classes) 的 one-hot。
        这里先将 one-hot 转成整型类别索引，再更新混淆矩阵。
        """
        y_true_argmax = tf.argmax(y_true, axis=-1)
        y_pred_argmax = tf.argmax(y_pred, axis=-1)

        y_true_argmax = tf.reshape(y_true_argmax, [-1])
        y_pred_argmax = tf.reshape(y_pred_argmax, [-1])

        current_cm = tf.math.confusion_matrix(
            y_true_argmax,
            y_pred_argmax,
            num_classes=self.num_classes,
            weights=sample_weight,
            dtype=tf.float32
        )
        self.total_cm.assign_add(current_cm)

    def result(self):
        # mean IoU = average over (true_positives / union)
        sum_over_row = tf.reduce_sum(self.total_cm, axis=0)
        sum_over_col = tf.reduce_sum(self.total_cm, axis=1)
        true_positives = tf.linalg.diag_part(self.total_cm)
        denominator = sum_over_row + sum_over_col - true_positives
        iou = true_positives / denominator
        return tf.reduce_mean(iou)

    def reset_states(self):
        # 每个 epoch 前自动重置
        tf.keras.backend.batch_set_value(
            [(v, tf.zeros_like(v)) for v in self.variables]
        )


In [ ]:
# X_curr, X_prev, Y_labels 的第一维是 batch_size = (Nt-1),
# 每一帧 shape = (256,256,1)，标签 shape = (128,128,2) 或 (256,256,2)

# 1. 定义指数衰减的学习率调度
initial_lr = 0.16
decay_rate = 0.95
decay_steps = 1000  # 每 1000 个 step 衰减一次，可根据数据大小和训练步数调整

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True  # 若为 True，则每到达 decay_steps 后才跳变；False 表示连续光滑衰减
)

# 2. 定义优化器为 SGD，并使用 lr_schedule 作为学习率
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

# 3. 编译模型
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.16),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[
        'accuracy',
        OneHotMeanIoU(num_classes=2)  # 二分类示例
    ]
)

Different ways to fit the model

In [ ]:
from tensorflow.keras.utils import Sequence


class MyDataGenerator(Sequence):
    def __init__(self, pt_instance, batch_size, steps_per_epoch):
        """
        pt_instance: 一个你用来生成合成数据的实例，比如 Particle_Tracking_Training_Data(Nt)
        batch_size: 每个batch的大小
        steps_per_epoch: 每个epoch的 batch 数量
        """
        self.pt = pt_instance
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch

        # 第一次生成数据
        self.on_epoch_end()

    def __len__(self):
        # 每次 epoch 中的 batch 数量
        return self.steps_per_epoch

    def __getitem__(self, index):
        # 从 self.X_curr, self.X_prev, self.Y_labels 中取出第 index 个 batch
        start_i = index * self.batch_size
        end_i = start_i + self.batch_size

        batch_X_curr = self.X_curr[start_i:end_i]
        batch_X_prev = self.X_prev[start_i:end_i]
        batch_Y = self.Y_labels[start_i:end_i]

        return ((batch_X_curr, batch_X_prev), batch_Y)

    def on_epoch_end(self):
        """
        每个 epoch 结束后自动调用
        - 在这里可以重新生成一整套 (Nt-1) 帧的合成数据，
          或者打乱已有数据的顺序
        """
        # 例如，每次都重新生成 Nt 帧
        # 这里演示：Nt=100
        Nt = 100 ## number of frames for each video
        kappa = 0.1 ## standard deviation of background noise added to image
        a = 3 ## scale factor for the size of particle spots (not true size of particles)
        IbackLevel = 0.15 ## relative intensity of randomly generated background pattern; in (0, 1)
        Nparticles = 10 ## the number of particles (more => slower)
        sigma_motion = 2.3 ## the standard deviation for particle brownian motion; should be in (0, 10)

        # 假设你调用 pt(...) 来生成数据
        vid, labels, tracks = self.pt(kappa, a, IbackLevel, Nparticles, sigma_motion)   # 根据需要传参

        # 再把 vid/labels 构造成 X_curr, X_prev, Y_labels
        # (Nt-1, 256,256,1) ...
        X_curr = []
        X_prev = []
        Y_labels = []
        for i in range(1, Nt):
            curr = np.expand_dims(vid[i],   axis=-1)  # (256,256,1)
            prev = np.expand_dims(vid[i-1], axis=-1)
            X_curr.append(curr)
            X_prev.append(prev)
            # 假设 labels[i] is (128,128,2)
            Y_labels.append(labels[i])

        # 转为 numpy array => (Nt-1, 256,256,1) ...
        self.X_curr = np.stack(X_curr, axis=0)
        self.X_prev = np.stack(X_prev, axis=0)
        self.Y_labels = np.stack(Y_labels, axis=0)

        # 这里可以再打乱
        idx = np.arange(self.X_curr.shape[0])
        np.random.shuffle(idx)
        self.X_curr = self.X_curr[idx]
        self.X_prev = self.X_prev[idx]
        self.Y_labels = self.Y_labels[idx]

# 使用
batch_size = 4
steps_per_epoch = 25  # 假设 (Nt-1)=99, 99/4≈25
pt_instance = Particle_Tracking_Training_Data.Particle_Tracking_Training_Data(Nt=100)  # 你的数据生成类

train_gen = MyDataGenerator(pt_instance, batch_size=batch_size, steps_per_epoch=steps_per_epoch)

history = model.fit(
    train_gen,
    epochs=20
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:780: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 126ms/step - accuracy: 0.8833 - loss: 52886872064.0000 - mean_iou: 0.4436
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9965 - loss: 0.8133 - mean_iou: 0.4982
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9957 - loss: 0.8133 - mean_iou: 0.4979
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9964 - loss: 0.8133 - mean_iou: 0.4982
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9958 - loss: 0.8133 - mean_iou: 0.4979
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9959 - loss: 0.8133 - mean_iou: 0.4980
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9964 - loss: 0.8133 - mean_iou: 0.4982
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9957 - loss: 0.8133 - mean_iou: 0.4979
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9965 - loss: 0.8133 - mean_iou: 0.4982
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 

In [ ]:
Nt = 100 ## number of frames for each video
kappa = 0.1 ## standard deviation of background noise added to image
a = 3. ## scale factor for the size of particle spots (not true size of particles)
IbackLevel = 0.15 ## relative intensity of randomly generated background pattern; in (0, 1)
Nparticles = 10 ## the number of particles (more => slower)
sigma_motion = 2.3 ## the standard deviation for particle brownian motion; should be in (0, 10)

## you might consider randomizing some of these parameters when training a neural net

pt = Particle_Tracking_Training_Data.Particle_Tracking_Training_Data(Nt) ## create object instance
## you can 'call' the object as many times as you want
## in this example, we only generate one training example
vid, labels, tracks = pt(kappa, a, IbackLevel, Nparticles, sigma_motion)

x_test = []
x_p = []
y_test = []

for i in range(0, Nt):
    # 当前帧 shape (256,256,1)
    curr = tf.expand_dims(vid[i],   axis=-1)
    pre = tf.expand_dims(vid[i-1], axis=-1)

    x_test.append(curr)
    x_p.append(pre)

    y_test.append(labels[i])  # 对应当前帧的标签

x_test = tf.stack(x_test, axis=0)  # (Nt-1, 256,256,1)
x_p = tf.stack(x_p, axis=0)  # (Nt-1, 256,256,1)
y_test = tf.stack(y_test, axis=0)

In [ ]:
model.evaluate([x_test, x_p], y_test, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:780: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 259ms/step - accuracy: 0.9958 - loss: 0.8133 - mean_iou: 0.4979


[0.813261866569519, 0.9958270192146301, 0.49791350960731506]

In [ ]:
predictions = model.predict([x_test, x_p])

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 255ms/step


In [ ]:
first_pred = predictions[0]  # 形状变为 (128, 128, 2)
print("First prediction shape:", first_pred.shape)
print("First prediction data:\n", first_pred)

In [ ]:
@interact(t=(0, Nt-2, 1))
def plotfn(t=0, show_tracks=True):
    fig = figure(1, [14, 7])
    fig.add_subplot(131)
    imshow(vid[t], origin='lower')
    if show_tracks:
        plot(tracks[t, :, 0], tracks[t, :, 1], 'rx')
    xlim(-10, 265)
    ylim(-10, 265)

    fig.add_subplot(132)
    imshow(vid[t], origin='lower')
    imshow(labels[t, ..., 1], origin='lower')

    fig.add_subplot(133)
    imshow(vid[t], origin='lower')
    imshow(predictions[t, ..., 1], origin='lower')

interactive(children=(IntSlider(value=0, description='t', max=98), Checkbox(value=True, description='show_trac…

研究一下加权loss

In [ ]:
# Define loss function
from tensorflow.keras.utils import register_keras_serializable

#@register_keras_serializable()
def weighted_binary_crossentropy(y_true, y_pred):
    """
    Weighted binary crossentropy loss, where the weight for class 1 (particles)
    is 300 times the weight for class 0 (background).
    """
    # Define weight
    weight_for_0 = 1.0
    weight_for_1 = 300.0

    # Compute weight tensor, with the same shape as y_true
    weights = tf.where(tf.equal(y_true[..., 1], 1), weight_for_1, weight_for_0)

    # Calculate binary crossentropy loss
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Apply weights
    weighted_bce = weights * bce

    # Return the mean of the weighted loss
    return tf.reduce_mean(weighted_bce)

model_weighted = build_multi_kernel_conv_model()
model_weighted.compile(optimizer='adam', loss=weighted_binary_crossentropy, metrics=['accuracy', OneHotMeanIoU(num_classes=2)])

num_videos = 20
for i in range(num_videos):
    print(i)

    vid, labels, tracks = pt(kappa, a, IbackLevel, Nparticles, sigma_motion)

    X_curr = []
    X_prev = []
    Y_labels = []

    for i in range(1, Nt):
        # 当前帧 shape (256,256,1)
        curr = tf.expand_dims(vid[i],   axis=-1)
        # 前一帧 shape (256,256,1)
        prev = tf.expand_dims(vid[i-1], axis=-1)

        X_curr.append(curr)
        X_prev.append(prev)

        Y_labels.append(labels[i])  # 对应当前帧的标签

    X_curr = tf.stack(X_curr, axis=0)  # (Nt-1, 256,256,1)
    X_prev = tf.stack(X_prev, axis=0)  # (Nt-1, 256,256,1)
    Y_labels = tf.stack(Y_labels, axis=0)

    model.fit([X_curr,X_prev], labels, batch_size=32, epochs=15, validation_split=0.2)

In [ ]:
Nt = 100 ## number of frames for each video
kappa = 0.1 ## standard deviation of background noise added to image
a = 3. ## scale factor for the size of particle spots (not true size of particles)
IbackLevel = 0.15 ## relative intensity of randomly generated background pattern; in (0, 1)
Nparticles = 10 ## the number of particles (more => slower)
sigma_motion = 2.3 ## the standard deviation for particle brownian motion; should be in (0, 10)

## you might consider randomizing some of these parameters when training a neural net

pt = Particle_Tracking_Training_Data(Nt) ## create object instance
## you can 'call' the object as many times as you want
## in this example, we only generate one training example
vid, labels, tracks = pt(kappa, a, IbackLevel, Nparticles, sigma_motion)

x_test = []
x_p = []
y_test = []

for i in range(0, Nt):
    # 当前帧 shape (256,256,1)
    curr = tf.expand_dims(vid[i],   axis=-1)
    pre = tf.expand_dims(vid[i-1], axis=-1)

    x_test.append(curr)
    x_p.append(pre)

    y_test.append(labels[i])  # 对应当前帧的标签

x_test = tf.stack(x_test, axis=0)  # (Nt-1, 256,256,1)
x_p = tf.stack(x_p, axis=0)  # (Nt-1, 256,256,1)
y_test = tf.stack(y_test, axis=0)

model_weighted.evaluate([x_test, x_p], y_test, batch_size=32)

predictions = model_weighted.predict([x_test, x_p])

@interact(t=(0, Nt-2, 1))
def plotfn(t=0, show_tracks=True):
    fig = figure(1, [14, 7])
    fig.add_subplot(131)
    imshow(vid[t], origin='lower')
    if show_tracks:
        plot(tracks[t, :, 0], tracks[t, :, 1], 'rx')
    xlim(-10, 265)
    ylim(-10, 265)

    fig.add_subplot(132)
    imshow(vid[t], origin='lower')
    imshow(labels[t, ..., 1], origin='lower')

    fig.add_subplot(133)
    imshow(vid[t], origin='lower')
    imshow(predictions[t, ..., 1], origin='lower')